# Random Survival Forest Model

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GridSearchCV, StratifiedKFold

from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored, as_concordance_index_ipcw_scorer, as_cumulative_dynamic_auc_scorer, as_integrated_brier_score_scorer
from sksurv.util import Surv

import time
from datetime import datetime, timedelta
import joblib

In [3]:
training_data_imputed_df = pd.read_csv("training_data_imputed_simple_TRAIN.csv.gz") 
# Filter to 30 day window
survival_data_30d = training_data_imputed_df.loc[
    (training_data_imputed_df['survival_time'] < 30) &
    (training_data_imputed_df['survival_time'] > 0)
    ]
survival_data_30d = survival_data_30d.astype({"cdiff_survival_flag": bool})

In [4]:
# train_df, val_df, = train_test_split(survival_data_30d, test_size=0.2, random_state=0)

In [5]:
# X_train = train_df.drop(['cdiff_2d_flag', 'cdiff_7d_flag', 'cdiff_30d_flag','cdiff_label_elisa_2d','cdiff_survival_flag','survival_time'], axis=1)
# y_train = train_df[['cdiff_survival_flag', 'survival_time']]
# y_array_event_time_train = np.array(
#     list(zip(y_train['cdiff_survival_flag'], y_train['survival_time'])), 
#     dtype=[('status', bool), ('survival_in_days', float)]
# )
# X_val = val_df.drop(['cdiff_2d_flag', 'cdiff_7d_flag', 'cdiff_30d_flag', 'cdiff_survival_flag', 'survival_time'], axis=1)
# y_val = val_df[['cdiff_survival_flag', 'survival_time']]
# y_array_event_time_val = np.array(
#     list(zip(y_val['cdiff_survival_flag'], y_val['survival_time'])), 
#     dtype=[('status', bool), ('survival_in_days', float)]
# )

In [6]:
X_train_val = survival_data_30d.drop(['cdiff_2d_flag', 'cdiff_7d_flag', 'cdiff_30d_flag','cdiff_label_elisa_2d','cdiff_survival_flag','survival_time'], axis=1)
print(X_train_val.shape)
y_train_val = survival_data_30d[['cdiff_survival_flag', 'survival_time']]
print(y_train_val.shape)
y_array_event_time_train = Surv.from_dataframe(time = 'survival_time', event = 'cdiff_survival_flag', data = y_train_val)

(83035, 170)
(83035, 2)


In [7]:
cdiff_X_train, cdiff_X_test, cdiff_y_train, cdiff_y_test = train_test_split(
    X_train_val,
    y_array_event_time_train,
    stratify=y_array_event_time_train['cdiff_survival_flag'],
    test_size = 0.2,
    random_state=1
)
lower, upper = np.percentile(y_array_event_time_train['survival_time'], [10, 90])
cdiff_times = np.arange(lower, upper + 1)

In [8]:
start_time = time.time()
start_datetime = datetime.now()
print(f"Training started at: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")

print("Starting GridSearchCV to find optimal parameters...")
rsf = RandomSurvivalForest(
    min_samples_leaf=2, n_jobs=-1, random_state=0, verbose=1
)

cv = KFold(n_splits=3, shuffle=True, random_state=1)
cv_param_grid = {
    # "estimator__max_depth": [1, 5, 10, 20],
    # "estimator__n_estimators": [10, 20, 50, 100],
    # "estimator__min_samples_split": [20, 50]
    "estimator__max_depth": [1, 5],
    "estimator__n_estimators": [10, 50, 100],
    "estimator__min_samples_split": [20]
}

cv_cindex = GridSearchCV(
    as_concordance_index_ipcw_scorer(rsf, tau=cdiff_times[-1]),
    param_grid=cv_param_grid,
    cv=cv,
).fit(X_train, y_array_event_time_train)

best_model_rsf = cv_cindex.best_estimator_

best_model_rsf_params = cv_cindex.best_params_
print(f"Best cross-validation C-index score: {cv_cindex.best_score_:.4f}")
print(f"Best cross-validated parameters:\n {best_model_rsf_params}")
end_time = time.time()
end_datetime = datetime.now()

duration_seconds = end_time - start_time
duration = timedelta(seconds=duration_seconds)

print(f"Training finished at: {end_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total training time: {duration}")

Training started at: 2025-05-24 17:13:27
Starting GridSearchCV to find optimal parameters...


NameError: name 'X_train' is not defined

In [71]:
y_array_event_time_train[['cdiff_survival_flag']]

array([(False,), (False,), (False,), ..., (False,), (False,), (False,)],
      shape=(83035,), dtype={'names': ['cdiff_survival_flag'], 'formats': ['?'], 'offsets': [0], 'itemsize': 9})

In [9]:
start_time = time.time()
start_datetime = datetime.now()
print(f"Training started at: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")

print("Starting GridSearchCV to find optimal parameters...")
rsf = RandomSurvivalForest(
    min_samples_leaf=2, n_jobs=-1, random_state=0, verbose=1
)

cv = KFold(n_splits=3, shuffle=True, random_state=1)
cv_param_grid = {
    # "estimator__max_depth": [1, 5, 10, 20],
    # "estimator__n_estimators": [10, 20, 50, 100],
    # "estimator__min_samples_split": [20, 50]
    "estimator__max_depth": [10],
    "estimator__n_estimators": [5],
    "estimator__min_samples_split": [20]
}

cv_cindex = GridSearchCV(
    concordance_index_censored(
        rsf,
        event_indicator = y_array_event_time_train['cdiff_survival_flag'],
        
        event_time = y_array_event_time_train['survival_time']
    ),
    param_grid=cv_param_grid,
    cv=cv,
    verbose=1
).fit(X_train_val, y_array_event_time_train)

best_model_rsf = cv_cindex.best_estimator_

best_model_rsf_params = cv_cindex.best_params_
print(f"Best cross-validation C-index score: {cv_cindex.best_score_:.4f}")
print(f"Best cross-validated parameters:\n {best_model_rsf_params}")
end_time = time.time()
end_datetime = datetime.now()

duration_seconds = end_time - start_time
duration = timedelta(seconds=duration_seconds)

print(f"Training finished at: {end_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total training time: {duration}")

Training started at: 2025-05-24 17:13:36
Starting GridSearchCV to find optimal parameters...


TypeError: concordance_index_censored() got multiple values for argument 'event_indicator'

In [48]:
rsf_best = RandomSurvivalForest(
    **best_model_rsf_params, n_jobs=-1, random_state=0
)
rsf_best.fit(X_train_val, y_array_event_time_train)

TypeError: RandomSurvivalForest.__init__() got an unexpected keyword argument 'estimator__max_depth'

In [16]:
joblib.dump(best_model_rsf, 
            f'{datetime.now().strftime("%Y-%m-%d_%H:%M")}_rsf_bestmodel_{round(cv_cindex.best_score_, 4)}.joblib')

['2025-05-24_16:41_rsf_bestmodel.joblib']

In [59]:
print(f'{datetime.now().strftime("%Y-%m-%d_%H:%M")}_rsf_bestmodel_{round(cv_cindex.best_score_, 4)}.joblib')




2025-05-24_17:03_rsf_bestmodel_0.9101.joblib


# Test Model!

In [1]:
loaded_model = joblib.load('2025-05-24_16:41_rsf_bestmodel.joblib')


NameError: name 'joblib' is not defined

In [ ]:
best_model_rsf_params

In [60]:
best_model_rsf

as_concordance_index_ipcw_scorer(estimator=RandomSurvivalForest(max_depth=1,
                                                                min_samples_leaf=2,
                                                                min_samples_split=20,
                                                                n_estimators=5,
                                                                n_jobs=-1,
                                                                random_state=0,
                                                                verbose=1),
                                 tau=np.float64(21.524583333333332))

In [74]:
test_data_loaded = pd.read_csv("training_data_imputed_simple_TRAIN.csv.gz") 
# Filter to 30 day window
survival_data_30d_finaltest = test_data_loaded.loc[
    (training_data_imputed_df['survival_time'] < 30) &
    (training_data_imputed_df['survival_time'] > 0)
    ]
survival_data_30d_finaltest = survival_data_30d_finaltest.astype({"cdiff_survival_flag": bool})

In [75]:
X_finaltest = survival_data_30d_finaltest.drop(['cdiff_2d_flag', 'cdiff_7d_flag', 'cdiff_30d_flag','cdiff_label_elisa_2d','cdiff_survival_flag','survival_time'], axis=1)
print(X_finaltest.shape)
y_finaltest = survival_data_30d_finaltest[['cdiff_survival_flag', 'survival_time']]
print(y_finaltest.shape)
y_array_event_time_finaltest = Surv.from_dataframe(time = 'survival_time', event = 'cdiff_survival_flag', data = y_finaltest)

(83035, 170)
(83035, 2)


In [ ]:
c_index_finaltest = loaded_model.score(X_finaltest, y_array_event_time_finaltest)
c_index_finaltest

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
